In [1]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
import keras_preprocessing
from keras_preprocessing import image
from keras_preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping
from keras_tuner.tuners import RandomSearch

train_dir = r'E:\AI\dataset_skeleton_sep\side\BicycleCrunch\training'
val_dir = r'E:\AI\dataset_skeleton_sep\side\BicycleCrunch\validation'
test_dir = r'E:\AI\dataset_skeleton_sep\side\BicycleCrunch\test'

# ImageDataGenerator 초기화
datagen = ImageDataGenerator(rescale=1./255)  # 이미지를 0과 1 사이의 값으로 정규화

# 훈련, 검증, 테스트 데이터셋을 위한 제너레이터 생성
train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical',
    shuffle=True)

validation_generator = datagen.flow_from_directory(
    val_dir,
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical',
    shuffle=False)

test_generator = datagen.flow_from_directory(
    test_dir,
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical',
    shuffle=False)


Found 7775 images belonging to 8 classes.
Found 1670 images belonging to 8 classes.
Found 1667 images belonging to 8 classes.


In [2]:
base_model = tf.keras.applications.resnet_v2.ResNet50V2(weights="imagenet",
                                                         include_top=False, 
                                                         input_shape=(128, 128, 3))
for layer in base_model.layers:
    base_model.trainable = False
for layer in base_model.layers[-9:]:
    base_model.trainable = True

model = models.Sequential(name="ResNet50v2_RT")
model.add(base_model)

model.add(layers.Flatten())
model.add(layers.Dense(256, activation = 'relu'))
model.add(layers.Dropout(0.2))
model.add(layers.Dense(512, activation = 'relu'))
model.add(layers.Dropout(0.2))
model.add(layers.Dense(512, activation = 'relu'))
model.add(layers.Dropout(0.2))
model.add(layers.Dense(8, activation = 'softmax')) 

model.summary()

Model: "ResNet50v2_RT"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50v2 (Functional)     (None, 4, 4, 2048)        23564800  
                                                                 
 flatten (Flatten)           (None, 32768)             0         
                                                                 
 dense (Dense)               (None, 256)               8388864   
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 512)               131584    
                                                                 
 dropout_1 (Dropout)         (None, 512)               0         
                                                                 
 dense_2 (Dense)             (None, 512)             

In [3]:

early_stopping = EarlyStopping(monitor='val_loss', patience=5, verbose=1)

model.compile(optimizer = optimizers.Adam(learning_rate = 0.00020752196203010584),  
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

history = model.fit(train_generator,
                    steps_per_epoch=train_generator.samples // train_generator.batch_size,
                    validation_data=validation_generator, 
                    validation_steps=validation_generator.samples // validation_generator.batch_size,
                    epochs = 25, 
                    callbacks=[early_stopping])

test_loss, test_acc = model.evaluate(test_generator, steps=test_generator.samples // test_generator.batch_size)
print('\n테스트 정확도:', test_acc)

Epoch 1/25
242/242 [==============================] - 38s 130ms/step - loss: 2.0980 - accuracy: 0.1406 - val_loss: 2.0169 - val_accuracy: 0.1911
Epoch 2/25
242/242 [==============================] - 18s 73ms/step - loss: 1.8988 - accuracy: 0.2261 - val_loss: 1.8749 - val_accuracy: 0.2506
Epoch 3/25
242/242 [==============================] - 18s 74ms/step - loss: 1.6576 - accuracy: 0.3141 - val_loss: 1.6144 - val_accuracy: 0.3462
Epoch 4/25
242/242 [==============================] - 18s 75ms/step - loss: 1.4342 - accuracy: 0.4058 - val_loss: 1.4185 - val_accuracy: 0.4357
Epoch 5/25
242/242 [==============================] - 18s 74ms/step - loss: 1.2470 - accuracy: 0.4936 - val_loss: 1.3855 - val_accuracy: 0.4736
Epoch 6/25
242/242 [==============================] - 18s 73ms/step - loss: 1.0690 - accuracy: 0.5852 - val_loss: 1.2769 - val_accuracy: 0.5132
Epoch 7/25
242/242 [==============================] - 18s 73ms/step - loss: 0.8679 - accuracy: 0.6709 - val_loss: 1.2928 - val_accuracy